In [36]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import pickle
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [37]:
test=pd.read_csv("test.csv")

In [38]:
X=test


In [39]:
X["Fare"].fillna(X['Fare'].mode()[0], inplace=True)

In [40]:
X["Age"].fillna(X['Age'].mode()[0], inplace=True)

In [41]:
X["Cabin"].fillna('No', inplace=True)

In [42]:
X["Embarked"].fillna('No', inplace=True)

In [43]:
X['familymem'] = X['SibSp'] + X['Parch'] + 1

In [44]:
X['alone'] = X['familymem'].map(lambda x: 1 if x<2 else 0)

In [45]:
X.drop(['Name', 'PassengerId', 'Ticket'], inplace=True, axis=1)

In [46]:
scale = ["Age", "Fare"]

In [47]:
skew_feets=X[scale].skew().sort_values(ascending=False)

In [48]:
skewness=pd.DataFrame({'Skew':skew_feets.astype('float')})

In [49]:
skewness=skewness[(skewness.Skew>.75)]

In [50]:
indeces = list(skewness.index)

In [51]:
for x in indeces:
    X[x] = np.log1p(X[x])

In [52]:
Xscale=X[scale]

In [53]:
scaler = pickle.load(open("scaler_titanic_pickle.pickle","rb"))

In [54]:
Xscale=pd.DataFrame(scaler.transform(Xscale), columns=Xscale.columns)

In [55]:
encode=list(set(X.columns)-set(scale)-set(['Id']))

In [56]:
X.isnull().sum().sum()

0

In [57]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Pclass     418 non-null    int64  
 1   Sex        418 non-null    object 
 2   Age        418 non-null    float64
 3   SibSp      418 non-null    int64  
 4   Parch      418 non-null    int64  
 5   Fare       418 non-null    float64
 6   Cabin      418 non-null    object 
 7   Embarked   418 non-null    object 
 8   familymem  418 non-null    int64  
 9   alone      418 non-null    int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 32.8+ KB


In [58]:
Xencode = X[encode]

In [59]:
X=Xscale.merge(Xencode.reset_index(), left_index=True, right_index=True)

In [60]:
X=pd.DataFrame(pd.get_dummies(data=X))

In [63]:
X

,Age,Fare,index,SibSp,Pclass,familymem,alone,Parch,Sex_female,Sex_male,...,Cabin_E60,Cabin_F,Cabin_F E46,Cabin_F E57,Cabin_F G63,Cabin_F2,Cabin_F33,Cabin_F4,Cabin_G6,Cabin_No
0,0.039577,0.348997,0,0,3,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,0.043368,0.333195,1,1,3,2,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0.046785,0.379604,2,0,2,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,0.036595,0.363449,3,0,3,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0.034123,0.414494,4,1,3,3,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.033564,0.352955,413,0,3,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
414,0.041077,0.753026,414,0,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
415,0.040919,0.338125,415,0,3,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
416,0.033564,0.352955,416,0,3,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [66]:
columns=pickle.load(open("columnssaved_titanic.pickle","rb"))
columns

Index(['Age', 'Fare', 'index', 'SibSp', 'alone', 'Parch', 'Pclass',
       'familymem', 'Cabin_A10', 'Cabin_A14',
       ...
       'Cabin_F4', 'Cabin_G6', 'Cabin_No', 'Cabin_T', 'Embarked_C',
       'Embarked_No', 'Embarked_Q', 'Embarked_S', 'Sex_female', 'Sex_male'],
      dtype='object', length=162)

In [69]:
X=X.reindex(columns=columns, fill_value=0)
X

,Age,Fare,index,SibSp,alone,Parch,Pclass,familymem,Cabin_A10,Cabin_A14,...,Cabin_F4,Cabin_G6,Cabin_No,Cabin_T,Embarked_C,Embarked_No,Embarked_Q,Embarked_S,Sex_female,Sex_male
0,0.039577,0.348997,0,0,1,0,3,1,0,0,...,0,0,1,0,0,0,1,0,0,1
1,0.043368,0.333195,1,1,0,0,3,2,0,0,...,0,0,1,0,0,0,0,1,1,0
2,0.046785,0.379604,2,0,1,0,2,1,0,0,...,0,0,1,0,0,0,1,0,0,1
3,0.036595,0.363449,3,0,1,0,3,1,0,0,...,0,0,1,0,0,0,0,1,0,1
4,0.034123,0.414494,4,1,0,1,3,3,0,0,...,0,0,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.033564,0.352955,413,0,1,0,3,1,0,0,...,0,0,1,0,0,0,0,1,0,1
414,0.041077,0.753026,414,0,1,0,1,1,0,0,...,0,0,0,0,1,0,0,0,1,0
415,0.040919,0.338125,415,0,1,0,3,1,0,0,...,0,0,1,0,0,0,0,1,0,1
416,0.033564,0.352955,416,0,1,0,3,1,0,0,...,0,0,1,0,0,0,0,1,0,1


In [70]:
model = pickle.load(open("logisticmodel.pickle","rb"))

In [78]:
preds=model.predict(X)

In [79]:
new_test= pd.read_csv('test.csv')

In [80]:
submission = pd.DataFrame(new_test['PassengerId'], columns=['PassengerId'])

In [81]:
submission['Survived'] = preds

In [82]:
submission.to_csv('submission1_6.csv', index=False, header=True)